### Main notebook

In [1]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# read data from csv
cpds = pd.read_csv('../GNN_toxic/data/raw/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)

# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    is_toxic = row['toxic']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_toxic)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

In [2]:
# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

# nodes: 8591 
# edges: 30026


  1%|▏         | 385/30026 [00:00<00:19, 1495.94it/s][15:52:02] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:52:02] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:52:02] WARNING: not removing hydrogen atom without neighbors
  3%|▎         | 848/30026 [00:00<00:19, 1507.86it/s][15:52:02] WARNING: not removing hydrogen atom without neighbors
[15:52:02] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:52:02] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:52:02] WARNING: not removing hydrogen atom with dummy atom neighbors
 11%|█▏        | 3413/30026 [00:02<00:24, 1070.07it/s][15:52:04] WARNING: not removing hydrogen atom without neighbors
[15:52:04] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:52:04] WARNING: not removing hydrogen atom with dummy atom neighbors
[15:52:04] WARNING: not removing hydrogen atom without neighbors
 13%|█▎        | 3893/30026 [00:02<00:19, 1330.77it/s][15:52:04] WARN

In [ ]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))

paths = graph.validate(test_cases, 'mol_weight')

In [ ]:
print(test_cases.iloc[20]['Pathway '])
print(test_cases.iloc[42]['Pathway '])
print(test_cases.iloc[47]['Pathway '])

In [ ]:
1/0

In [8]:
p, smiles_smi, idx, comm_changes, idxcom = graph.constrained_shortest_path('C00082', 'C01533', weight='mol_weight')
for i in p: print(i)

['C00082', 'C00355', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C05604', 'C00355', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00811', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00355', 'C04045', 'C22038', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C05604', 'C00355', 'C04045', 'C22038', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00355', 'C10447', 'C01197', 'C01494', 'C02666', 'C12204', 'C05610', 'C02325', 'C01533']


In [11]:
comm_changes

[6, 8, 5, 6, 8, 4]

In [5]:
smiles_smi

[2.122612655363154,
 4.140081434759724,
 2.0447306848705313,
 2.8136602705518063,
 4.831129049948376,
 2.1491555363099826]

### Study graph

### Try cluster graph nodes

### Try similarity based on SMILES

In [ ]:
smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C00323').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))


smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C12096').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))